In [11]:
from bs4 import BeautifulSoup
import requests
import json


with open('assets/lesson_topics.json', 'r', encoding='utf-8') as f:
    lessonTopics = json.load(f)

questions_collection = []

with open('assets/topics_questions.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)

for url, topics_map in lessonTopics.items():
    lesson_id = url.split('/').pop().split('?')[0]
    categoryId = url.split('/').pop().split('?s=')[1].split('&')[0]
    category = int(categoryId) + 3
    for topicName, subTopicList in topics_map.items():
        for subTopic in subTopicList:
            subTopicId = subTopic['value']
            subTopicName = subTopic['text']
            indexOfTopicInTopicsMap = list(topics_map.keys()).index(topicName)
            for questionUrl, questionData in questions_data.items():
                if questionUrl == (f"https://ogmmateryal.eba.gov.tr/api/soru-tur-listele?id={subTopicId}"):
                    for question in questionData['questions']:
                        questionDoc = {
                            "topicPath": f"category/{category}/lessons/{lesson_id}/topics/{indexOfTopicInTopicsMap}/subtopics/{subTopicId}",
                            "id": question['id'],
                            "answer": question['cevap'],
                            "url": f"https://ogmmateryal.eba.gov.tr/soru-bankasi/{lesson_id}/test?s={categoryId}&k={subTopicId}&id={question['id']}&t=css&ks=0&os=0&zs=0"
                        }

                        questions_collection.append(questionDoc)


In [29]:
import time
firebaseQuestions = []


# Create and open processed questions file


for url, topics_map in lessonTopics.items():
    
    lesson_id = url.split('/').pop().split('?')[0]
    categoryId = url.split('/').pop().split('?s=')[1].split('&')[0]
    # Create file name using lesson_id and categoryId
    file_name = f"{lesson_id}_{categoryId}_questions.json"
    
    # Initialize empty list for questions
    questionsAsDict = []
    
    # Open file in write mode
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(questionsAsDict, f, ensure_ascii=False)
    for question in questions_collection:
        if question['url'].startswith(f"https://ogmmateryal.eba.gov.tr/soru-bankasi/{lesson_id}/test?s={categoryId}"):
            # Print progress
            print(f"{questions_collection.index(question) + 1}/{len(questions_collection)}", end='\r')
            # Get question HTML from URL with retry logic and delay
            max_retries = 3
            retry_delay = 2  # seconds

            for attempt in range(max_retries):
                try:
                    response = requests.get(question['url'])
                    if response.status_code == 200:
                        break
                    elif response.status_code == 429:  # Too Many Requests
                        if attempt < max_retries - 1:  # Don't sleep on last attempt
                            time.sleep(retry_delay)
                            retry_delay *= 2  # Exponential backoff
                        continue
                    else:
                        response.raise_for_status()
                except requests.exceptions.RequestException as e:
                    if attempt < max_retries - 1:
                        time.sleep(retry_delay)
                        retry_delay *= 2
                        continue
                    print(
                        f"Failed to get question {question['id']} after {max_retries} attempts: {e}")
                    continue

            soup = BeautifulSoup(response.text, 'html.parser')
            question_html = soup.find('div', class_='question-item')
            if question_html:
                question["questionAsHtml"] = str(question_html)
                texts = question_html.stripped_strings
                question_text = ' '.join(texts)
                question["questionText"] = question_text

                if question["questionText"].startswith(f"Soru 1: {question['id']}"):
                    question["questionText"] = question["questionText"].split(
                        f"Soru 1: {question['id']}")[-1].strip()

                firebaseQuestions.append(question)
                questionsAsDict.append(question)
                with open(file_name, 'w', encoding='utf-8') as f: 
                    json.dump(questionsAsDict, f, ensure_ascii=False)

            # Add delay between requests to avoid rate limiting
            time.sleep(1)

    len(firebaseQuestions)







In [8]:
filePaths = [
    'biyoloji_6_questions.json',
    'biyoloji_7_questions.json',
    'biyoloji_8_questions.json', 
    'biyoloji_9_questions.json',
    'cografya_6_questions.json',
    'cografya_7_questions.json',
    'cografya_8_questions.json',
    'cografya_9_questions.json',
    'dikab_6_questions.json',
    'dikab_7_questions.json',
    'dikab_8_questions.json',
    'dikab_9_questions.json',
    'felsefe_7_questions.json',
    'felsefe_8_questions.json',
    'fizik_6_questions.json',
    'fizik_7_questions.json', 
    'fizik_8_questions.json',
    'fizik_9_questions.json',
    'fl-biyoloji_6_questions.json',
    'fl-biyoloji_7_questions.json',
    'fl-biyoloji_8_questions.json',
    'fl-biyoloji_9_questions.json',
    'fl-fizik_6_questions.json',
    'fl-fizik_7_questions.json',
    'fl-fizik_8_questions.json',
    'fl-fizik_9_questions.json',
    'fl-kimya_6_questions.json',
    'fl-kimya_7_questions.json',
    'fl-kimya_8_questions.json',
    'fl-kimya_9_questions.json',
    'fl-matematik_6_questions.json',
    'fl-matematik_7_questions.json',
    'fl-matematik_8_questions.json',
    'fl-matematik_9_questions.json',
    'ingilizce_6_questions.json',
    'ingilizce_7_questions.json',
    'ingilizce_8_questions.json',
    'ingilizce_9_questions.json',
    'inkilap-tarihi_9_questions.json',
    'kimya_6_questions.json',
    'kimya_7_questions.json',
    'kimya_8_questions.json',
    'kimya_9_questions.json',
    'matematik_4_questions.json',
    'matematik_6_questions.json',
    'matematik_7_questions.json',
    'matematik_8_questions.json',
    'matematik_9_questions.json',
    'tarih_6_questions.json',
    'tarih_7_questions.json',
    'tarih_8_questions.json',
    'tde_6_questions.json',
    'tde_7_questions.json',
    'tde_8_questions.json',
    'tde_9_questions.json',
]

import json
stats = {}

for filename in filePaths:
    with open(filename, 'r', encoding='utf-8') as f:
        questions = json.load(f)
        
        # Initialize counters
        image_count = 0
        no_image_count = 0
        total_questions = len(questions)
        
        for question in questions:
            # Check for href tags and add withImage flag
            has_image = 'questionAsHtml' in question and 'img' in question['questionAsHtml'].lower()
            question['withImage'] = has_image
            
            if has_image:
                image_count += 1
            else:
                no_image_count += 1
                
        # Save the modified questions back to file
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(questions, f, ensure_ascii=False, indent=2)
                
        # Add stats to dict if there are any href tags
        if image_count > 0:
            stats[filename] = {
                'total_questions': total_questions,
                'image_count': image_count,
                'no_image_count': no_image_count
            }

print("File statistics:")
for file, file_stats in stats.items():
    print(f"\n{file}:")
    print(f"  Total questions: {file_stats['total_questions']}")
    print(f"  Questions with images: {file_stats['image_count']}")
    print(f"  Questions without images: {file_stats['no_image_count']}")


# Calculate total questions with no images across all files
total_no_image = sum(file_stats['no_image_count'] for file_stats in stats.values())
print(f"\nTotal questions without images across all files: {total_no_image}")


File statistics:

biyoloji_6_questions.json:
  Total questions: 1099
  Questions with images: 827
  Questions without images: 272

biyoloji_7_questions.json:
  Total questions: 875
  Questions with images: 614
  Questions without images: 261

biyoloji_8_questions.json:
  Total questions: 1506
  Questions with images: 1050
  Questions without images: 456

biyoloji_9_questions.json:
  Total questions: 976
  Questions with images: 530
  Questions without images: 446

cografya_6_questions.json:
  Total questions: 649
  Questions with images: 269
  Questions without images: 380

cografya_7_questions.json:
  Total questions: 730
  Questions with images: 229
  Questions without images: 501

cografya_8_questions.json:
  Total questions: 1195
  Questions with images: 394
  Questions without images: 801

cografya_9_questions.json:
  Total questions: 582
  Questions with images: 240
  Questions without images: 342

dikab_6_questions.json:
  Total questions: 225
  Questions with images: 94
  Quest

In [10]:
import firebase_admin
from firebase_admin import credentials, firestore
from tqdm import tqdm
from google.cloud import firestore
from google.oauth2 import service_account

# Initialize credentials from service account
creds = service_account.Credentials.from_service_account_file('firebase_credentials.json')

# Initialize Firestore client with service account
db = firestore.Client(credentials=creds)

# Create a batch writer
def batch_write(db, docs, batch_size=500):
    batch = db.batch()
    count = 0
    
    for doc in tqdm(docs):
        # Create document reference with question ID
        doc_ref = db.collection('questions').document(str(doc['id']))
        batch.set(doc_ref, doc)
        count += 1
        
        # When batch_size is reached, commit and reset
        if count == batch_size:
            batch.commit()
            batch = db.batch()
            count = 0
    
    # Commit any remaining documents
    if count > 0:
        batch.commit()

# Write questions in batches
batch_write(db, firebaseQuestions)
print(f"Successfully wrote {len(firebaseQuestions)} questions to Firestore")


100%|██████████| 10/10 [00:00<00:00, 18657.94it/s]


Successfully wrote 10 questions to Firestore
